In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
model_path='/home/manojale/Documents/Performance_Dataset/New_project_proposal/Nxcode-CQ-7B-orpo/'

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(92416, 4096)
    (layers): ModuleList(
      (0-31): 32 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear(in_features=4096, out_features=512, bias=True)
          (v_proj): Linear(in_features=4096, out_features=512, bias=True)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=4096, out_features=13440, bias=False)
          (up_proj): Linear(in_features=4096, out_features=13440, bias=False)
          (down_proj): Linear(in_features=13440, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Line

In [2]:
import pandas as pd
samples=pd.read_csv('Data/Verified_Samples.csv')

In [5]:
samples

,Unnamed: 0,commit_url,commit_message,diff_before,diff_after,diff,pred_primary,pred_secondary,Quality
0,0,https://github.com/fangfufu/Linux-Fake-Backgro...,Removing frame.flags.writeable = False Removin...,"def compose_frame(self, frame):\n frame.flags...","def compose_frame(self, frame):\n mask = self...","@@ -292,7 +292,6 @@ then scale & crop the imag...",Inefficient_Algorithm/Data-structure,Expensive_Operation,good
1,1,https://github.com/kugelrund/mesh_segmentation...,Improve performance of calculating graph lapla...,"def segment_mesh(mesh, k, coefficients, action...","def segment_mesh(mesh, k, coefficients, action...","@@ -248,9 +248,9 @@ def segment_mesh(mesh, k, ...",Inefficient_Algorithm/Data-structure,Expensive_Operation,good
2,3,https://github.com/zama-ai/concrete-ml/commit/...,chore: improve performances of numpy_gemm clos...,"def numpy_gemm(\n a: numpy.ndarray,\n b: num...","def numpy_gemm(\n a: numpy.ndarray,\n b: num...","@@ -117,7 +117,15 @@ def numpy_gemm(\n b_prim...",Inefficient_Algorithm/Data-structure,Expensive_Operation,good
3,4,https://github.com/urule99/jsunpack-n/commit/c...,pdf regex replaced with character processing t...,def applyFilter(input):\n output = re.sub('^[...,def applyFilter(input):\n if len(input) > 100...,"@@ -497,8 +497,29 @@ class pdf:\n \n @static...",Inefficient_Algorithm/Data-structure,Expensive_Operation,good
4,6,https://github.com/bayesianbandits/bayesianban...,MAINT: improve performance by removing unneces...,"def decay(self, X: NDArray[Any]) -> None:\n ""...","def decay(self, X: NDArray[Any]) -> None:\n ""...","@@ -606,14 +606,12 @@ class NormalRegressor(Ba...",Inefficient_Algorithm/Data-structure,Expensive_Operation,good
5,7,https://github.com/pygae/clifford/commit/36b59...,Improve performance of arithmetic on MultiVect...,"def _checkOther(self, other, coerce=True) -> T...","def _checkOther(self, other, coerce=True) -> T...","@@ -61,7 +61,13 @@ class MultiVector(object):\...",Inefficient_Algorithm/Data-structure,Expensive_Operation,good
6,8,https://github.com/optuna/optuna/commit/dd3ede...,Make set_trial_param() of RedisStorage faster ...,"def set_trial_param(\n self,\n trial_id: int...","def set_trial_param(\n self,\n trial_id: int...","@@ -1,3 +1,4 @@\n +from collections.abc import...",Inefficient_I/O,Inefficient_Disk_I/O,good
7,9,https://github.com/DingGuodong/LinuxBashShellS...,improve performance and cut down time user wait,"def getServiceName(self, name):\n if isinstan...","def getServiceName(self, name):\n if isinstan...","@@ -84,9 +84,8 @@ class remoteWindowsWMI(objec...",Inefficient_I/O,Inefficient_Disk_I/O,good
8,10,https://github.com/bartfeenstra/betty/commit/4...,Link instead of copy rendered files to improve...,"def _filter_file(site: Site, file: File) -> st...","def _filter_file(site: Site, file: File) -> st...","@@ -266,7 +266,7 @@ def _filter_file(site: Sit...",Inefficient_I/O,Inefficient_Disk_I/O,good
9,11,https://github.com/arangodb/python-arango/comm...,[PERF] improve performance of large batch commits,"def commit(self):\n """"""Execute the queued API...","def commit(self):\n """"""Execute the queued API...","@@ -104,13 +104,14 @@ class BatchExecution(Con...",Inefficient_I/O,Inefficient_Disk_I/O,good


In [3]:
import pandas as pd
import os
temp_df=samples

batched_predictions = []

# Your provided prompt template
prompt_template = '''
You are provided with a GitHub commit in this format:
Commit Message: """the commit message written by the author"""
Original Code: """the code before the code modificaton"""
Modified Code: """the code after the code modification"""
Code Diff : """the diff between the original and the modified code"""

The commit is implementing a code optimization to improve software performance or resource utilization.
"
Your task is to meticulously examine the commit message, the original code, the modified code, and the differences between the original and modified code (code diff) to understand the improvements in terms of  performance or resource usage. Focus primarily on the changes made to the code as well as the commit message to grasp how they improve performance or resource usage. Your goal is to understand:"

1) The root cause of inefficiency in the original code that necessitated these optimizations.
2) The optimization implemented in the revised code, including a deeper understanding on how the modification addresses the identified inefficiencies.

Your output should be structured to clearly present:
Root Cause of Inefficiency: Explain the specific performance or resource inefficiency issues found in the original code and their root causes.
Optimization Strategy: Based on the provided commit and modified code, elaborate the strategies to overcome the root cause of the inefficiencies, detailing the rationale and significance of each optimization. Explain the optimization such a way that one can implement the code modification by following the strategy. Use the "related notes" section to better explain the commit if required.

- Commit Message: 
{commit_message}

- Original Code:
```{Original_Code}```

- Modified Code:
```{Modified_Code}```

Code Diff:
```{Code_Diff}```

Model Response: '''

for i in range(len(samples)):
    print('Processing {}'.format(i))
    original_code = temp_df.loc[i, 'diff_before']
    modified_code = temp_df.loc[i, 'diff_after']
    commit_message = temp_df.loc[i, 'commit_message']
    code_diff=temp_df.loc[i,'diff']
    
    prompt = prompt_template.format(Original_Code=original_code, Modified_Code=modified_code, commit_message=commit_message,Code_Diff= code_diff)
    
    # Assuming the `model` and `tokenizer` are set up for generating responses based on the prompt
    inputs = tokenizer.apply_chat_template(
            [{'role': 'user', 'content': prompt}],
            return_tensors="pt"
        ).to(model.device)
    
    outputs = model.generate(
        inputs, 
        max_new_tokens=2046,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    analysis = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    
    batched_predictions.append(analysis)

Processing 0
Processing 1
Processing 2
Processing 3
Processing 4
Processing 5
Processing 6
Processing 7
Processing 8
Processing 9
Processing 10
Processing 11
Processing 12
Processing 13
Processing 14
Processing 15
Processing 16
Processing 17
Processing 18
Processing 19
Processing 20
Processing 21
Processing 22
Processing 23
Processing 24
Processing 25
Processing 26
Processing 27
Processing 28
Processing 29
Processing 30


In [4]:
temp_df['pl_nl']=batched_predictions

In [5]:
temp_df.isnull().sum()

Unnamed: 0        0
commit_url        0
commit_message    0
diff_before       0
diff_after        0
diff              0
pred_primary      0
pred_secondary    0
Quality           0
pl_nl             0
dtype: int64

In [6]:
conversion_prompt_template = '''
You are provided with an explanation of a code optimization and the original code. Your task is to generate the optimized code based on the explanation. Here are the details:

Explanation: 
{explanation}

Original Code:
```{original_code}```

Your task is to generate the optimized code that implements the given explanation. The optimized code should reflect the improvements described in the explanation.

Optimized Code: '''

# List to store the new code predictions
new_code_predictions = []

# Process each explanation to generate the optimized code
for i in range(len(temp_df)):
    print('Converting explanation to code for sample {}'.format(i))
    explanation = temp_df.loc[i, 'pl_nl']
    original_code = temp_df.loc[i, 'diff_before']
    
    prompt = conversion_prompt_template.format(explanation=explanation, original_code=original_code)
    
    # Generate the optimized code based on the explanation
    inputs = tokenizer.apply_chat_template(
        [{'role': 'user', 'content': prompt}],
        return_tensors="pt"
    ).to(model.device)
    
    outputs = model.generate(
        inputs,
        max_new_tokens=2046,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    optimized_code = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    
    new_code_predictions.append(optimized_code)

# Store the optimized code in a new column in the dataframe
temp_df['optimized_code'] = new_code_predictions


Converting explanation to code for sample 0
Converting explanation to code for sample 1
Converting explanation to code for sample 2
Converting explanation to code for sample 3
Converting explanation to code for sample 4
Converting explanation to code for sample 5
Converting explanation to code for sample 6
Converting explanation to code for sample 7
Converting explanation to code for sample 8
Converting explanation to code for sample 9
Converting explanation to code for sample 10
Converting explanation to code for sample 11
Converting explanation to code for sample 12
Converting explanation to code for sample 13
Converting explanation to code for sample 14
Converting explanation to code for sample 15
Converting explanation to code for sample 16
Converting explanation to code for sample 17
Converting explanation to code for sample 18
Converting explanation to code for sample 19
Converting explanation to code for sample 20
Converting explanation to code for sample 21
Converting explanati

In [8]:
print(temp_df['optimized_code'][1])

Here's an example of what the optimized code could look like in Python:

```python
import numpy as np
from scipy.sparse import diags
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel

def segment_mesh(mesh, k, coefficients, action):
    """Segments the given mesh into k clusters using RBF K-means and performs the given action for each cluster
    """

    # set coefficients
    global delta
    global eta
    delta, eta = coefficients

    # affinity matrix
    W = rbf_kernel(mesh)
    print("mesh_segmentation: Calculating graph laplacian...")
    
    # graph laplacian
    row_sums = W.sum(1)
    Dsqrt = diags(np.power(row_sums, -0.5).tolist()[0])
    L = Dsqrt.dot(W).dot(Dsqrt)
    
    print("mesh_segmentation: Calculating eigenvectors...")
    
    # get eigenvectors
    _, V = np.linalg.eigh(L)
    
    print("mesh_segmentation: Preparing kmeans...")
    
    # compute association matrix
    Q = V.dot(V.transpose())
    Q = rbf_kernel(Q)

    # app

In [12]:
print(temp_df['pl_nl'][1])


Root Cause of Inefficiency: The root cause of inefficiency in the original code lies in the computation of the graph laplacian. The original implementation first computes the degree matrix and then applies it to the affinity matrix to obtain the graph laplacian. This leads to high computational costs for large matrices.

Optimization Strategy: 
1. Improved Efficiency: To improve performance, the modified code calculates the degree matrix and the graph laplacian in a more efficient manner. It first computes the square root of the reciprocal of the sum of entries in each row of the affinity matrix, and then multiplies the original affinity matrix by the degree matrices and then transpose-multiplies them to obtain the graph laplacian. This significantly reduces the computational complexity of the computation. 
2. Simplified Code: The code diff clearly shows the simplification of the code by using numpy broadcasting operations. This reduces the number of lines of code and makes the code e

In [28]:
print(temp_df['diff_before'][1])

def segment_mesh(mesh, k, coefficients, action):
  """Segments the given mesh into k clusters and performs the given
  action for each cluster
  """
 
  # set coefficients
  global delta
  global eta
  delta, eta = coefficients
 
  # affinity matrix
  W = _create_affinity_matrix(mesh)
  print("mesh_segmentation: Calculating graph laplacian...")
  # degree matrix
  Dsqrt = numpy.diag([math.sqrt(1/entry) for entry in W.sum(1)])
  # graph laplacian
  L = Dsqrt.dot(W.dot(Dsqrt))
 
  print("mesh_segmentation: Calculating eigenvectors...")
  # get eigenvectors
  l,V = scipy.linalg.eigh(L, eigvals = (L.shape[0] - k, L.shape[0] - 1))
  # normalize each column to unit length
  V = V / [numpy.linalg.norm(column) for column in V.transpose()]
 
  print("mesh_segmentation: Preparing kmeans...")
  # compute association matrix
  Q = V.dot(V.transpose())
  # compute initial guess for clustering
  initial_clusters = _initial_guess(Q, k)
 
  print("mesh_segmentation: Applying kmeans...")
  # apply kmean

In [29]:
print(temp_df['diff_after'][1])

def segment_mesh(mesh, k, coefficients, action):
  """Segments the given mesh into k clusters and performs the given
  action for each cluster
  """
 
  # set coefficients
  global delta
  global eta
  delta, eta = coefficients
 
  # affinity matrix
  W = _create_affinity_matrix(mesh)
  print("mesh_segmentation: Calculating graph laplacian...")
  # degree matrix
  Dsqrt = numpy.sqrt(numpy.reciprocal(W.sum(1)))
  # graph laplacian
  L = ((W * Dsqrt).transpose() * Dsqrt).transpose()
 
  print("mesh_segmentation: Calculating eigenvectors...")
  # get eigenvectors
  l,V = scipy.linalg.eigh(L, eigvals = (L.shape[0] - k, L.shape[0] - 1))
  # normalize each column to unit length
  V = V / [numpy.linalg.norm(column) for column in V.transpose()]
 
  print("mesh_segmentation: Preparing kmeans...")
  # compute association matrix
  Q = V.dot(V.transpose())
  # compute initial guess for clustering
  initial_clusters = _initial_guess(Q, k)
 
  print("mesh_segmentation: Applying kmeans...")
  # apply

In [30]:
temp_df['commit_url'][1]

'https://github.com/kugelrund/mesh_segmentation/commit/d97ba5446e96ef6cab24bd41f9dbf655f4a26a1f'

In [7]:
temp_df.to_csv('Nxcode-CQ-7B-orpo_results.csv',index=False)